In [1]:
!pip install pandas numpy openai

In [2]:
import pandas as pd
import numpy as np

In [3]:
contents = []

for i in range(1, 8):
    file_name = f"./control_points/{i}.txt"
    try:
        with open(file_name, 'r', encoding="utf-8") as file:
            content = file.read()
            contents.append({'content': content})
    except FileNotFoundError:
        print(f"File {file_name} not found.")

df = pd.DataFrame(contents, columns=['content'])

In [5]:
import re

In [6]:
pd.options.mode.chained_assignment = None 
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df['content']= df["content"].apply(lambda x : normalize_text(x))

In [7]:
import os
from openai import AzureOpenAI

In [8]:
os.environ["AZURE_OPENAI_KEY"] = "7e93421f46cd4680831023addcb0f42d"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://francecentral-openai.openai.azure.com"
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_KEY"),  
  api_version = "2023-05-15",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

def generate_embeddings(text, model="ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df['embedding'] = df["content"].apply(lambda x : generate_embeddings (x, model = "ada-002")) 

In [9]:
df.to_json("control_points.json", orient="records")